# Hectoc

In [ ]:
%pip install numpy
import numpy as np

In [2]:
# Initialisierungen

# Mögliche Operatoren
possible_operators = ['+','-','*','**', '']

# Zahlenpaare, die eingeklammert werden können
bracket_pairs = np.array([
    [[0],[1]],[[0],[2]],[[0],[3]],[[0],[4]],
    [[1],[2]],[[1],[3]],[[1],[4]],[[1],[5]],
    [[2],[3]],[[2],[4]],[[2],[5]],
    [[3],[4]],[[3],[5]],
    [[4],[5]]])

# Startzustand: keine Operatoren, keine Klammern
s_start = ['','','','','','0','0','0','0','0','0','0','0','0','0','0','0','0','0']

# Ziel des arithmetischen Ausdrucks
goal = 100

In [3]:
# Erstelle den arithmetischen Ausdruck im aktuellem Status
def get_arithmetic_expression(numbers, operators):
    expression = ''
    last_char_type = None
    for number_id in range(len(numbers)):
        possible_brackets = list(np.where(bracket_pairs==number_id)[0])
        for bracket_id in possible_brackets:
            if int(operators[len(numbers)-1+bracket_id]) == 1 and bracket_pairs[bracket_id][0]==number_id:
                if last_char_type == "num":
                    return None
                expression += '('
                last_char_type = "open"
        if last_char_type == "close":
            return None
        expression += str(numbers[number_id])
        last_char_type = "num"
        for bracket_id in possible_brackets:
            if int(operators[len(numbers)-1+bracket_id]) == 1 and bracket_pairs[bracket_id][1]==number_id:
                expression += ')'
                last_char_type = "close"
        if number_id != len(numbers)-1 and operators[number_id] != '':
            expression += str(operators[number_id])
            last_char_type = "op"
    return expression

In [4]:
# Berechne Wert des arithmetischen Ausdrucks im aktuellem Status
def calculate_result(numbers, operators):
    expression = get_arithmetic_expression(numbers, operators)
    if expression is None:
        return None
    result = eval(expression)
    return result

In [5]:
# Printe den arithmetischen Ausdruck und den entsprechenden Wert des aktuellen Status
def print_result(numbers, operators):
    expression = get_arithmetic_expression(numbers, operators)
    if expression is None:
        print("Invalid expression.")
        return
    result = eval(expression)
    print(expression + " = " + str(result))

In [6]:
# Prüfe ob der Status zulässig ist
def applicable(state, numbers):
    # Prüfe ob Wert zu groß für Speicher
    if len(np.where(np.array(state) == "**")[0]) > 1:
        return False 
    # Prüfe ob Klammern sich überschneiden
    brackets = np.where(np.array(state)=='1')[0] - len(numbers)+1
    if len(brackets) > 1:
        for bracket_1 in brackets:
            for bracket_2 in brackets:
                if (bracket_pairs[bracket_1][0] < bracket_pairs[bracket_2][0] 
                    and bracket_pairs[bracket_2][0] <= bracket_pairs[bracket_1][1] 
                    and bracket_pairs[bracket_1][1] < bracket_pairs[bracket_2][1]):
                    return False
                if (bracket_pairs[bracket_2][0] < bracket_pairs[bracket_1][0] 
                    and bracket_pairs[bracket_1][0] <= bracket_pairs[bracket_2][1] 
                    and bracket_pairs[bracket_2][1] < bracket_pairs[bracket_1][1]):
                    return False
    # Prüfe ob arithmetischer Ausdruck syntaktisch korrekt ist
    try:
        return calculate_result(numbers, state) is not None
    except (ZeroDivisionError,SyntaxError,TypeError,OverflowError,RuntimeError):
        return False

In [7]:
# Breitensuche wie in der Vorlesung
def bfs(s_start, numbers, goal):
    tree_level=0
    itera = 0
    visited = []
    queue = [(s_start, tree_level)]
    while len(queue)>0:
        (s,depth) = queue.pop(0)  # Immer das erste Element aus der Queue betrachten: FIFO Prinzip
        if depth < len(numbers)-1:  # Füge zuerst Operatoren ein
            A = possible_operators
        else:                       # Falls notwendig, ergänze Klammern
            A = np.where(np.array(s)=='0')[0]
        for a in A:
            s_n = s.copy()
            if depth < len(numbers)-1: 
                s_n[depth] = a
            else:
                s_n[a] = '1'
            if applicable(s_n, numbers):
                if calculate_result(numbers, s_n) == goal:
                    return s_n
                if depth<len(s_start)-1:
                    queue.append((s_n, depth+1))
    print('Keine Lösung!')
    return None

## Teste das Programm:

In [8]:
#315281
numbers = [3,1,5,2,8,1]
result_s = bfs(s_start, numbers, goal)
print_result(numbers, result_s)

3+(1+5)*2*8+1 = 100


In [9]:
#479113
numbers = [4,7,9,1,1,3]
result_s = bfs(s_start, numbers, goal)
print_result(numbers, result_s)

4+7+91+1-3 = 100


In [10]:
#681132
numbers = [6,8,1,1,3,2]
result_s = bfs(s_start, numbers, goal)
print_result(numbers, result_s)

68+1-1+32 = 100


### Alternative: Erzwinge zwischen jeder Ziffer einen Operator != ''

In [11]:
# Mögliche Operatoren, kein '' mehr möglich
possible_operators = ['+','-','*','**']
# Startzustand: initialer Operator, keine Klammern
s_start = ['+','+','+','+','+','0','0','0','0','0','0','0','0','0','0','0','0','0','0']

In [12]:
#315281
numbers = [3,1,5,2,8,1]
result_s = bfs(s_start, numbers, goal)
print_result(numbers, result_s)

3+(1+5)*2*8+1 = 100


In [13]:
#479113
numbers = [4,7,9,1,1,3]
result_s = bfs(s_start, numbers, goal)
print_result(numbers, result_s)

(4+7)*9+1*1**3 = 100


In [14]:
#681132
numbers = [6,8,1,1,3,2]
result_s = bfs(s_start, numbers, goal)
print_result(numbers, result_s)

(6+8-1-1*3)**2 = 100
